## **Token classification :**
- assigns a label to each indivial token in a sentence
- one of the most common token classification is Name Entity Recognition(NER)
- NER attempt to find a label for each entity in a sentence , such as person, organization, event, location , company etc

- Finetune **DistilBERT** on the WNUT 17 dataset to detect new entities

**Login with huggingface community, so that we can upload our finetune model to huggingface community**

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

## **Load WNUT 17 dataset**

**Dataset Summary**
WNUT 17: Emerging and Rare entity recognition

This shared task focuses on identifying unusual, previously-unseen entities in the context of emerging discussions. Named entities form the basis of many modern approaches to other tasks (like event clustering and summarisation), but recall on them is a real problem in noisy text - even among annotators. This drop tends to be due to novel entities and surface forms. **Take for example the tweet “so.. kktny in 30 mins?”** - even human experts find entity kktny hard to detect and resolve. This task will evaluate the ability to **detect and classify novel, emerging, singleton named entities in noisy text**.

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
wnut = load_dataset('wnut_17')


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [ ]:
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

Each number in **ner_tags** represents an entity.

In [ ]:
wnut['train'][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

**Convert each numbers in the ner_tags convert to their label names to find out what the entities are**

In [ ]:
label_list = wnut['train'].features[f'ner_tags'].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

- B- indicates the beginning of an entity.
- I- indicates a token is contained inside the same entity (for example, the - - State token is a part of an entity like Empire State Building).
- 0 indicates the token doesn’t correspond to any entity.

## **Preprocess**
Load DistillBERT tokenizer to preprocesst the token fields

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

**As you saw in the example tokens field above, it looks like the input has already been tokenized. But the input actually hasn’t been tokenized yet and you’ll need to set is_split_into_words=True to tokenize the words into subwords. For example:**

In [ ]:
example = wnut['train'][0]
tokenzied_input = tokenizer(example['tokens'], is_split_into_words=True)                 # every model having their own tokenizer, tokenizer return "input_ids" and "attention_mask"
print(tokenzied_input)
tokens = tokenizer.convert_ids_to_tokens(tokenzied_input['input_ids'])

{'input_ids': [101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


- However, this adds some special tokens [CLS] and [SEP] and the subword tokenization creates a mismatch between the input and labels.
- A single word corresponding to a single label may now be split into two subwords. You’ll need to realign the tokens and labels by:

**1:** Mapping all tokens to their corresponding word with the **word_ids** method.<br>
**2:** Assigning the label -100 to the special tokens [CLS] and [SEP] so they’re ignored by the PyTorch loss function (see CrossEntropyLoss).<br>
**3:** Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.<br>

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT’s maximum input length:

In [ ]:
tokens

In [ ]:
# with the help of these function we align and labels
def tokenize_and_align_labels(examples):      # preprocessing function
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Apply the preprocessing function over the entire dataset use dataset_map function,
# you can speed up the map function be setting
# batch=True to process multiple elements of the dataset at once

tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Now create a batch of examples using DataCollatorWithPadding. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
print("data_collator: ", data_collator)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
print("data_collator: ", data_collator)

## **Evaluate**
Including a metric during training is often helpful for evaluating your model’s performance. You can quickly load a evaluation method with the 🤗 Evaluate library. For this task, load the seqeval framework (see the 🤗 Evaluate quick tour to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [ ]:
import evaluate
seqeval = evaluate.load('seqeval')

Get the NER labels first, and then create a function that passes your true predictions and true labels to compute to calculate the scores:

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## **Train**<br>
Before you start training your model, create a map of the expected ids to their labels with id2label and label2id:

In [ ]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [ ]:
from transformers import AutoModelForTokenClassification , TrainingArguments , Trainer

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels = 13, id2label=id2label, label2id=label2id)

- Define your training hyperparameters in TrainingArguments. The only required parameter is output_dir which specifies where to save your model. You’ll push this model to the Hub by setting push_to_hub=True (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the Trainer will evaluate the seqeval scores and save the training checkpoint.
- Pass the training arguments to Trainer along with the model, dataset, tokenizer, data collator, and compute_metrics function.
- Call train() to finetune your model.

In [ ]:
training_argm = TrainingArguments(
    output_dir = "my_ner_model",
    learning_rate = 2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy='epochs',
    save_strategy='epochs',
    load_best_model_at_end=True,
    push_to_hub=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_argm,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Once training is completed, share your model to the Hub with the push_to_hub() method so everyone can use your model:

In [ ]:
trainer.push_to_hub()